# In-Vehicle Coupon Recommendation: Data Cleaning Pipeline  
**Goal**: Clean and preprocess raw coupon recommendation data for machine learning.  
**Key Steps**:  
- Handle missing values  
- Standardize categorical features  
- Encode variables (One-Hot)  
- Scale numerical features  
**Outputs**:  
- `coupon_data_cleaned.csv` (raw cleaned data)  
- `coupon_data_cleaned_encoded.csv` (ML-ready data)  

## 1. Load Data  
- **File**: `in-vehicle-coupon-recommendation.csv`  
- **Preview**: First 5 rows below.  

In [54]:
# Importing required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Loading the raw dataset
df = pd.read_csv('../Data/in-vehicle-coupon-recommendation.csv')

# Displaying the first 5 rows to get a quick overview
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


## 2. Explore Data Structure  
- **Key Columns**:  
  - Categorical: `destination`, `coupon`, `gender`...  
  - Numerical: `temperature`, `toCoupon_GEQ5min`...  
- **Missing Values**: Check next output.  

In [55]:
# Checking general information about the dataset
df.info()

# Identifying categorical and numerical columns
categorical_columns = df.select_dtypes(include='object').columns.tolist()
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

## 3. Handle Missing Data  
- **Drop `car`**: 99% missing → Irrecoverable.  
- **Impute Others**:  
  - Categorical → Mode (most frequent value).  
  - Example: `CoffeeHouse` NaN → "never".  

In [56]:
# Check for missing values
missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0]

car                     12576
CoffeeHouse               217
Restaurant20To50          189
CarryAway                 151
RestaurantLessThan20      130
Bar                       107
dtype: int64

In [57]:
# Drop columns with too many missing values (threshold = 40% of total rows)
threshold = len(df) * 0.4
cols_to_drop = missing[missing > threshold].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)

# Fill missing values in categorical columns with the mode — safer version
for col in df.select_dtypes(include='object'):
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()[0]
        df[col] = df[col].fillna(mode_val)

In [58]:
# Columns with frequency-style categorical values
freq_columns = ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']

# Mapping for standardizing categorical values
standard_map = {
    'less1': 'Less than 1',
    'gt8': 'More than 8'
}

# Apply standardization to each relevant column
for col in freq_columns:
    if col in df.columns:
        df[col] = df[col].replace(standard_map)


In [71]:
df.to_csv('../Data/coupon_data_cleaned.csv', index=False)  # Cleaned but NOT encoded

## 4. Encode & Scale Features  
- **One-Hot Encoding**: For 17 categorical columns.  
- **Scaling**: `StandardScaler` on `temperature`, `toCoupon_GEQ5min`, etc.  

In [72]:
# List of categorical columns to encode (excluding 'car' since it's mostly missing)
categorical_to_encode = [
    'destination', 'passanger', 'weather', 'time', 'coupon',
    'expiration', 'gender', 'age', 'maritalStatus',
    'education', 'occupation', 'income',
    'Bar', 'CoffeeHouse', 'CarryAway',
    'RestaurantLessThan20', 'Restaurant20To50'
]

# One-hot encoding of selected categorical features
df_encoded = pd.get_dummies(df, columns=categorical_to_encode, drop_first=True)

In [73]:
# Numerical columns to scale (excluding binary columns)
numerical_to_scale = ['temperature', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min']

# caling numerical features
scaler = StandardScaler()
df_encoded[numerical_to_scale] = scaler.fit_transform(df_encoded[numerical_to_scale])

In [74]:
# Saving the cleaned, encoded and scaled dataset
df_encoded.to_csv('../Data/coupon_data_cleaned_encoded.csv', index=False)